In [106]:
import requests
import re
from bs4 import BeautifulSoup as BS
from collections import Counter
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from pprint import pprint

In [2]:
# def scrape_text(url):
#     response = requests.get(url)
#     return response.text


# def parse_raw(content):
#     return BS(content, 'lxml')

In [90]:
url = r'https://erikamentari.wordpress.com/2018/02/27/jre-1070-jordan-peterson-transcript/'

content = requests.get(url).text
soup = BS(content, 'lxml')
paragraphs = soup.find_all('p')
paragraphs = [p.get_text().replace('Joe Rogan: ', 'Joe: ').replace('Dr Jordan B Peterson: ', "Jordan: ")\
              for p in paragraphs[8:-10]]
paragraph_text = ' '.join(paragraphs)
print(len(paragraphs))
print(len(paragraph_text))

611
163379


In [77]:
names = ['Joe Rogan', 'Joe', 'Dr Jordan B Peterson', 'Jordan']
paragraphs[-1]
for i, p in enumerate(paragraphs):
    if p == unlabeled[-1]:
        print(paragraphs[i-2:i+2])

['Jordan: Yeah. Well, even deeper— that the right narrative is, the way that we should view the world is victim vs. oppressor. Because that’s the basic postmodern, neo-Marxist template. It’s that the right way to view the world is that it’s a power ground, it’s a battleground of power interests competing constantly. The ones that win are oppressors, the ones who lose are oppressed. That’s the way to look at the world and I think that that’s wrong, that’s a bad way of looking at the world, psychologically, sociologically, politically, economically, ideologically, you name it. It ends in nothing but catastrophe. I mean, first of all because it puts your group identity as something that’s paramount. And I mean that’s just not— well, that isn’t what we do in the West, let’s say; we put your individual identity paramount. And then, well that’s just for starters fundamentally, and then— I guess the other reason that people are on my case to some degree is because I have made a strong case, w

In [101]:
rogan_p = [p.replace('Joe:', '').strip() for p in paragraphs if p.startswith('Joe:')]
peterson_p = [p.replace('Jordan:', '').strip() for p in paragraphs if p.startswith('Jordan:')]
unlabeled = [p for p in paragraphs if not p.startswith('Joe:') and not p.startswith('Jordan:')]
rogan_p.insert(1, unlabeled.pop(0))
print(len(rogan_p))
print(len(peterson_p))
print(unlabeled)

306
304
['18:18']


In [102]:
rogan_words = ' '.join(rogan_p).split(' ')
peterson_words = ' '.join(peterson_p).split(' ')
rogan_count, peterson_count = len(rogan_words), len(peterson_words)
total_count = rogan_count + peterson_count
rogan_percent = rogan_count / total_count
peterson_percent = peterson_count / total_count
print('Total word count:', total_count)
print('Rogan word count:', rogan_count)
print('Peterson word count:', peterson_count)
print('% of conversation (Rogan): {}%'.format(round(100* rogan_percent, 2)))
print('% of conversation (Peterson): {}%'.format(round(100 * peterson_percent, 2)))

Total word count: 29119
Rogan word count: 6736
Peterson word count: 22383
% of conversation (Rogan): 23.13%
% of conversation (Peterson): 76.87%


In [103]:
Counter(rogan_words).most_common
Counter(peterson_words).most_common

<bound method Counter.most_common of Counter({'the': 868, 'to': 614, 'and': 524, 'you': 517, 'of': 498, 'a': 493, 'that': 466, 'I': 430, 'is': 291, 'in': 262, 'And': 244, 'it': 219, 'it’s': 195, 'like,': 189, 'that’s': 181, 'It’s': 174, 'what': 168, 'they': 156, 'have': 149, 'so': 144, 'was': 135, 'for': 134, 'people': 133, 'be': 128, 'do': 125, 'your': 124, 'because': 117, 'well,': 117, 'like': 113, 'not': 110, 'this': 107, 'then': 103, 'you’re': 102, 'are': 99, 'with': 95, 'going': 94, 'about': 94, 'on': 89, 'but': 89, 'think': 87, 'at': 87, 'all': 84, 'I’m': 84, 'if': 82, 'out': 81, 'don’t': 81, 'Well,': 78, 'we': 70, 'can': 69, 'just': 69, 'as': 68, 'know': 67, 'You': 66, 'there’s': 66, 'up': 65, 'an': 64, 'when': 63, 'Yeah,': 61, 'more': 61, 'So': 61, 'it.': 61, 'things': 59, 'That’s': 59, 'how': 59, 'want': 59, 'that.': 58, 'they’re': 57, 'get': 56, 'my': 56, 'know,': 54, 'there': 53, 'one': 53, 'So,': 51, 'way': 49, 'something': 49, 'who': 48, 'But': 48, 'say': 48, 'so,': 48, 'h

In [105]:
rogan_corpus = ' '.join(rogan_words)
peterson_corpus = ' '.join(peterson_words)
rogan_word_toke, rogan_p_toke = word_tokenize(rogan_corpus), sent_tokenize(rogan_corpus)
peterson_word_toke, peterson_p_toke = word_tokenize(peterson_corpus), sent_tokenize(peterson_corpus)

In [107]:
stop_words = stopwords.words('english')
rogan_word_toke = [w for w in rogan_word_toke if w not in stop_words]
peterson_word_toke = [w for w in peterson_word_toke if w not in stop_words]